In [1]:
from utils.load_env_vars import load_env
load_env()

from torch.utils.data import DataLoader
from data.dataloader import CyListSceneUnderstandingDataset, CyListVQADataset
import json
device ='cuda:2'

-----------------------------
Environment variables loaded:
HF_HOME /pfss/mlde/workspaces/mlde_wsp_PI_Kersting/LLaVA-cake/Cyclist/hfcache
HUGGINGFACE_HUB_CACHE /pfss/mlde/workspaces/mlde_wsp_PI_Kersting/LLaVA-cake/Cyclist/hfcache
WANDB_PROJECT llava-changes
-----------------------------


# datalaoder

In [2]:
data_path="/pfss/mlde/workspaces/mlde_wsp_PI_Kersting/LLaVA-cake/Cyclist/output"
cyc_dataset = CyListSceneUnderstandingDataset(data_path)

torch_dataset = DataLoader(cyc_dataset, batch_size=2, shuffle=True)

#print one example
for i, data in enumerate(torch_dataset):
    print(data[0].shape)
    print(type(data[1][0]))
    print(data[1][0])
    break

torch.Size([2, 42, 1080, 1920, 3])
<class 'str'>
[
    {
        "shape": "cube",
        "size": "small",
        "material": "metal",
        "color": "purple",
        "transformation": {
            "enlarges": "False",
            "rotates": "False",
            "orbits": "False",
            "change_colors": "False"
        }
    },
    {
        "shape": "cone",
        "size": "small",
        "material": "metal",
        "color": "cyan",
        "transformation": {
            "enlarges": "True",
            "rotates": "False",
            "orbits": 0,
            "change_colors": "blue"
        }
    },
    {
        "shape": "cylinder",
        "size": "small",
        "material": "metal",
        "color": "blue",
        "transformation": {
            "enlarges": "False",
            "rotates": "True",
            "orbits": "False",
            "change_colors": "False"
        }
    }
]


# model

In [3]:
import requests
from PIL import Image

import torch
from data.model_loader import load_model_and_processor

model_id = "llava-hf/llava-onevision-qwen2-72b-ov-chat-hf"
model_id = "llava-hf/llava-onevision-qwen2-7b-ov-chat-hf"

model, processor = load_model_and_processor(model_id, device=device)



OpenCLIP not installed


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


# Eval loop
load different models and aggregate them in model loader file

In [5]:
query = """
Hey, can you describe the scene in the video?
"""


for i, data in enumerate(torch_dataset):
    videos = data[0] #[BS, T, H, W, C] -> [[2, 42, 1080, 1920, 3]) 
    scene_json = data[1]
    
    
    # processor accepts tensors with channel [T, H, W,C] -> (20, 240, 320, 3)

    
    #produce prompts
    prompts = []
    for _ in scene_json:
        # Each "content" is a list of dicts and you can add image/video/text modalities
        conversation = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": query},
                    {"type": "video"},
                    ],
            },
        ]
        prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
        prompts.append(prompt)


    inputs = processor(text=prompts, videos=torch.unbind(videos, dim=0), padding=True, return_tensors="pt").to(model.device, torch.float16)

    # Generate the output tokens
    generate_kwargs = {"max_new_tokens": 500, "do_sample": False, "top_p": 0.9}
    output = model.generate(**inputs, **generate_kwargs)

    # Find the eos token id in the inputs
    eos_token_id = processor.tokenizer.eos_token_id
    eos_token_pos = torch.where(inputs["input_ids"] == eos_token_id)[1]

    # Extract only the newly generated tokens
    new_tokens = []
    for i, eos_pos in enumerate(eos_token_pos):
        new_tokens.append(output[i, eos_pos + 1:])

    # Decode the newly generated tokens
    generated_text = processor.batch_decode(new_tokens, skip_special_tokens=True)
    print("True scene json")
    for scene in scene_json:
        print(scene) 
    print("Generated scene json")
    for text in generated_text:
        print(text)
    break

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


True scene json
[
    {
        "shape": "cylinder",
        "size": "large",
        "material": "metal",
        "color": "red",
        "transformation": {
            "enlarges": "False",
            "rotates": "False",
            "orbits": "False",
            "change_colors": "gray"
        }
    },
    {
        "shape": "cylinder",
        "size": "small",
        "material": "rubber",
        "color": "gray",
        "transformation": {
            "enlarges": "False",
            "rotates": "False",
            "orbits": 0,
            "change_colors": "False"
        }
    },
    {
        "shape": "cone",
        "size": "small",
        "material": "rubber",
        "color": "red",
        "transformation": {
            "enlarges": "True",
            "rotates": "False",
            "orbits": "False",
            "change_colors": "False"
        }
    },
    {
        "shape": "cylinder",
        "size": "small",
        "material": "rubber",
        "color": "yellow",
 